In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/pRoBERTa/data

/content/drive/.shortcut-targets-by-id/1LF-WP2ZCgIIdJNo5hvCwikjO_3USvDrU/pRoBERTa/data


In [3]:
# Cài đặt thư viện underthesea
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 47.3 MB/s eta 0:00:00


In [4]:
warmup = "/content/drive/MyDrive/pRoBERTa/data/ise-dsc01-warmup.json"
train = '/content/drive/MyDrive/pRoBERTa/data/ise-dsc01-train.json'
public_test = '/content/drive/MyDrive/pRoBERTa/data/ise-dsc01-public-test-offcial.json'
private_test = "/content/drive/MyDrive/pRoBERTa/data/ise-dsc01-private-test-offcial.json"

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os
import math
import re
from underthesea import word_tokenize
import torch
from underthesea import text_normalize
from tqdm import tqdm

## Preprocesss of DSC

In [6]:
def preprocess_text(text: str) -> str:
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text

# Preprocessing text
Quy trình thực hiện như sau:
* Chuẩn hóa bảng mã Unicode
* Chuẩn hóa kiểu gõ dấu tiếng Việt
* Thực hiện tách từ tiếng Việt
* Đưa về dạng viết thường
* Xóa các ký tự đặc biệt

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os
import math
import re
from underthesea import word_tokenize
import torch
from underthesea import text_normalize

# Chuẩn hóa Unicode và dấu câu
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

# Hàm tạo dict để convert định dạng cũ sang định dạng mới
def loaddicchar():
  dic = {}
  char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')
  charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')
  for i in range(len(char1252)):
      dic[char1252[i]] = charutf8[i]
  return dic

dicchar = loaddicchar()
def convert_unicode(text):
    text = re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], text)
    return text_normalize(text)

# Đưa về dạng viết thường
def to_lower_case(text):
    text = text.lower()
    return text


# Chuẩn hoá những từ lặp âm tiết
def remove_duplicate_characters(text):
    pattern = re.compile(r'(\w)\1{2,}')
    text = pattern.sub(r'\1', text)
    return text

# Tách từ tiếng Việt
def word_token(text):
    return word_tokenize(text, format='text')

# xử lý stopword
def remove_stopwords(text, path = "/content/drive/MyDrive/pRoBERTa/data/vietnamese-stopwords-dash.txt"):
    # stopwords = open(path)
    # stopwords = stopwords.readlines()
    # stopwords = [x.strip() for x in stopwords]
    stopwords = []
    with open(path, 'r', encoding='UTF-8') as file:
        for line in file:
            stopwords.append(line.strip())
    words = text.split(' ')
    res = list()
    for word in words:
        if word not in stopwords:
            res.append(word)

    return ' '.join(res)

In [8]:
def tonghop(text):
  # text = preprocess_text(text)
  # text = convert_unicode(text)
  # text = to_lower_case(text)
  # text = remove_duplicate_characters(text)
  text = word_token(text)
  # text = remove_stopwords(text)
  return text



# Chia context thành từng câu riêng lẽ

In [9]:
# # -*- coding: utf-8 -*-
# import re
# Academic_degree = "(TTƯT|PGS|TS|BS|ThS|CKII|CKI|GS)"
# Middle_Name = "(St|G|J|W|R)"
# alphabets= "([A-Za-z])"
# prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
# suffixes = "(Inc|Ltd|Jr|Sr|Co)"
# starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
# acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
# websites = "[.](com|net|org|io|gov|edu|me)"
# digits = "([0-9])"
# multiple_dots = r'\.{2,}'
# Upper_alphabets = "([A-Z])"


# def split_into_sentences(text: str) -> list[str]:
#     text = " " + text + "  "
#     text = text.replace("\n"," ")
#     text = text.replace("TP.", "TP<prd>")
#     text = text.replace("Tp.", "Tp<prd>")
#     text = text.replace("HCM.", "HCM<stop>")
#     text = re.sub(prefixes,"\\1<prd>",text)
#     text = re.sub(websites,"<prd>\\1",text)
#     text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
#     text = re.sub(r"\.\.\.\s([A-Z])", r"...<stop>\1", text)
#     text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
#     if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
#     text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
#     text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
#     text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
#     text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
#     text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
#     text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
#     text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
#     text = re.sub(Academic_degree + "[.]", " \\1<prd>",text)
#     text = re.sub(Middle_Name + "[.]", " \\1<prd>",text)
#     text = re.sub(Upper_alphabets + "[.]", " \\1<prd>", text )
#     if "”" in text: text = text.replace(".”","”.")
#     if "\"" in text: text = text.replace(".\"","\".")
#     if "!" in text: text = text.replace("!\"","\"!")
#     if "?" in text: text = text.replace("?\"","\"?")
#     text = text.replace(".",".<stop>")
#     text = text.replace("?","?<stop>")
#     text = text.replace("!","!<stop>")
#     text = text.replace("<prd>",".")
#     sentences = text.split("<stop>")
#     sentences = [tonghop(s.strip()) for s in sentences]
#     if sentences and not sentences[-1]: sentences = sentences[:-1]
#     # sentences = re.split(r'(?<=[.!?…])\s+', text)
#     return sentences

Academic_degree = "(TTƯT|PGS|TS|BS|ThS|CKII|CKI|GS)"
Middle_Name = "(St)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'
alphabets= "([A-Za-z])"
Upper_alphabets = "(\s[a-z]{0,1}[A-Z]{1,2})"
infor_national = "(AI)"


def split_into_sentences(text: str) -> list[str]:
    text = " " + text + "  "
    # replace sentence contain \n\n latest to ""
    text = re.sub(r"\n\n(?!.*\n\n)", "<check>", text)
    sentences = text.split("<check>")

    if len(sentences[-1].split())<6:
        text = sentences[0]
    else:
        text = "\n\n".join(sentences)
    # convert stop with "dot + space"
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = text.replace("...\n\n", "...<stop> \n\n")
    text = text.replace("?\n\n", "?<stop> \n\n")
    text = text.replace("!\n\n", "!<stop> \n\n")
    text = text.replace("…\n\n", "…<stop> \n\n")

    text = re.sub(r"\.\.\.\s([A-Z])", r"...<stop>\1", text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)), text)

    #
    text = text.replace(r"\sTP.", "TP<prd>")
    text = text.replace(r"\sTp.", "Tp<prd>")

    text = text.replace(".\n\n", "<stop> \n\n")
    text = re.sub(Middle_Name + "[.]", " \\1<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]", "\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]", "\\1<prd>\\2<prd>",text)
    text = re.sub(alphabets + "[.]" + infor_national, "\\1<prd>\\2",text)
    text = re.sub(Academic_degree + "[.]", " \\1<prd>",text)
    # text = re.sub(infor_national + "[.]", " \\1<stop>",text)
    text = re.sub(Upper_alphabets + "[.]", " \\1<prd>", text )

    #
    text = text.replace(". ",".<stop>")
    text = text.replace("<prd>",".")

    # split sentence
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences


In [10]:
import json
def converted_data(data_link,name):
  # Đọc dữ liệu từ file JSON
  with open(data_link) as file:
      data = json.load(file)

  # Tạo một từ điển để lưu trữ dữ liệu đã chuyển đổi
  converted_data = {}

  # Lặp qua mỗi ID trong data
  for id, item in tqdm(data.items()):
      data = {}
      # Lấy giá trị trong key "context"
      context = item['context']
      # Phân tách thành mảng các câu
      sentences = split_into_sentences(context)
      data["context"] = sentences
      data["claim"] = item["claim"]
      if ("private-test" not in data_link) and ("public-test" not in data_link):
        data["verdict"] = item["verdict"]
        data["evidence"] = item["evidence"] if item["evidence"] != None else None
        data["domain"] = item["domain"]
      # Lưu trữ data vào từ điển theo ID tương ứng
      converted_data[id] = data

  # with open('converted_data.json', 'w') as file:
  #     json.dump(converted_data, file)

  with open(f"/content/drive/MyDrive/pRoBERTa/data/converted_data_{name}.json", "w", encoding="utf-8") as file:
      json.dump(converted_data, file, ensure_ascii=False)

In [40]:
a = {"context": "Từ 15h mỗi ngày, chị Trang, chủ hàng ốc bươu nhồi thịt Đà Lạt... đã có mặt tại khu ẩm thực... Hồ Thị Kỷ HOCMAI. để bày biện món, chuẩn bị bán cho thực khách mỗi tối.\n\nTừng con ốc nhồi đầy đặn, đặt trong chiếc xửng hấp nóng, dậy mùi thơm, điểm xuyết những trái ớt đỏ và tiêu xanh trông khá bắt mắt. Nhiều thực khách đi ngang hàng ốc này bị cuốn hút bởi cách bài trí và mùi thơm thường dừng lại thưởng thức.\n\nChủ hàng ốc cho biết là người miền Tây nhưng chị rất thích hương vị ốc bươu nhồi thịt kiểu Đà Lạt trong một lần đến xứ này. Vì vậy, sau khi trở lại Sài Gòn, chị đã quyết định mở hàng ốc nhồi, hơn một năm trước. \"Tôi may mắn vì đã học và làm thành công món ốc nhồi chuẩn hương vị Đà Lạt. Có lẽ đây cũng là lý do nhiều thực khách khen ngợi\", chị Trang chia sẻ.\n\nBên cạnh bày trí ấn tượng, từng con ốc nhồi ở hàng này còn ghi điểm nhờ phần thịt thơm, nhân chặt, dai. Chị Trang cho biết, để từng con ốc nhồi thơm ngon, đậm vị, công đoạn quan trọng nhất chính là làm nhân và nhồi thịt.\n\nPhần nhân, chị chuẩn bị các nguyên liệu gồm ốc bươu làm sạch, hấp cùng sả đến khi chín thì lấy thịt ra, rửa sạch và cắt nhỏ. Tiếp đến chị sẽ trộn thịt ốc cùng thịt heo băm, giò sống, nấm mèo, hành tím, tỏi, gừng, sả cùng các loại gia vị sao cho hài hòa. Để con ốc đẹp mắt, chị phải rửa thật sạch vỏ ốc và dùng một lá sả lót trước, sau đó, nhồi phần nhân vào vỏ ốc với một lượng vừa, phải thật chặt tay để thịt ốc không bị rơi ra và khi ăn có độ dai nhất định.\n\nTừng con ốc sau khi nhồi sẽ đem hấp trong chiếc xửng có thêm sả, sau 30 phút ốc chín, dậy mùi thơm. Ốc sẽ còn được hấp nóng suốt quá trình bán. Hiện, mỗi ngày, chị Trang nhận khoảng 80 kg ốc bươu từ mối quen ở Hậu Giang gửi lên và ngày nào cũng hết sạch. Số ốc này cùng những nguyên liệu khác sẽ luôn được dùng hết trong ngày để đảm bảo món ăn luôn tươi mới.\n\n\"Ốc nhồi trông tưởng đơn giản nhưng công đoạn chuẩn bị rất mất thời gian. Song tôi vui vì món ăn của mình rất hợp khẩu vị và làm hài lòng nhiều thực khách\", chủ hàng ốc nói.\n\nMón này ăn kèm có rau răm, lá tía tô, rau thơm và chấm với nước mắm gừng rất bắt vị. Hàng ốc sẽ bắt đầu mở bán từ 15h đến khoảng 21h hàng ngày. Mỗi con có giá 7.000 đồng.\n\nKhánh Thiện",
        "claim": "Do đó, chị đã quyết định mở hàng ốc nhồi, hơn một năm trước sau khi chị quay lại Sài Gòn",
        "verdict": "SUPPORTED",
        "evidence": "Vì vậy, sau khi trở lại Sài Gòn, chị đã quyết định mở hàng ốc nhồi, hơn một năm trước",
        "domain": "du-lich"
}

In [41]:
a["context"]

'Từ 15h mỗi ngày, chị Trang, chủ hàng ốc bươu nhồi thịt Đà Lạt... đã có mặt tại khu ẩm thực... Hồ Thị Kỷ HOCMAI. để bày biện món, chuẩn bị bán cho thực khách mỗi tối.\n\nTừng con ốc nhồi đầy đặn, đặt trong chiếc xửng hấp nóng, dậy mùi thơm, điểm xuyết những trái ớt đỏ và tiêu xanh trông khá bắt mắt. Nhiều thực khách đi ngang hàng ốc này bị cuốn hút bởi cách bài trí và mùi thơm thường dừng lại thưởng thức.\n\nChủ hàng ốc cho biết là người miền Tây nhưng chị rất thích hương vị ốc bươu nhồi thịt kiểu Đà Lạt trong một lần đến xứ này. Vì vậy, sau khi trở lại Sài Gòn, chị đã quyết định mở hàng ốc nhồi, hơn một năm trước. "Tôi may mắn vì đã học và làm thành công món ốc nhồi chuẩn hương vị Đà Lạt. Có lẽ đây cũng là lý do nhiều thực khách khen ngợi", chị Trang chia sẻ.\n\nBên cạnh bày trí ấn tượng, từng con ốc nhồi ở hàng này còn ghi điểm nhờ phần thịt thơm, nhân chặt, dai. Chị Trang cho biết, để từng con ốc nhồi thơm ngon, đậm vị, công đoạn quan trọng nhất chính là làm nhân và nhồi thịt.\n\nPh

In [42]:
split_into_sentences(a["context"])

['Từ 15h mỗi ngày, chị Trang, chủ hàng ốc bươu nhồi thịt Đà Lạt... đã có mặt tại khu ẩm thực...',
 'Hồ Thị Kỷ HOCMAI.',
 'để bày biện món, chuẩn bị bán cho thực khách mỗi tối',
 'Từng con ốc nhồi đầy đặn, đặt trong chiếc xửng hấp nóng, dậy mùi thơm, điểm xuyết những trái ớt đỏ và tiêu xanh trông khá bắt mắt.',
 'Nhiều thực khách đi ngang hàng ốc này bị cuốn hút bởi cách bài trí và mùi thơm thường dừng lại thưởng thức',
 'Chủ hàng ốc cho biết là người miền Tây nhưng chị rất thích hương vị ốc bươu nhồi thịt kiểu Đà Lạt trong một lần đến xứ này.',
 'Vì vậy, sau khi trở lại Sài Gòn, chị đã quyết định mở hàng ốc nhồi, hơn một năm trước.',
 '"Tôi may mắn vì đã học và làm thành công món ốc nhồi chuẩn hương vị Đà Lạt.',
 'Có lẽ đây cũng là lý do nhiều thực khách khen ngợi", chị Trang chia sẻ',
 'Bên cạnh bày trí ấn tượng, từng con ốc nhồi ở hàng này còn ghi điểm nhờ phần thịt thơm, nhân chặt, dai.',
 'Chị Trang cho biết, để từng con ốc nhồi thơm ngon, đậm vị, công đoạn quan trọng nhất chính là

In [14]:
# converted_data(warmup, "warmup_tocken")

In [15]:
# converted_data(train, "train_tocken")

In [16]:
converted_data(private_test, "private-test_split")

100%|██████████| 5396/5396 [00:03<00:00, 1424.82it/s]


In [17]:
import random
from torch.utils.data import Dataset
import json
import pandas as pd
class LoadDataFromJson(Dataset, ):
    def __init__(self, datasets):
        self.samples_pos = []
        self.samples_neg = []
        with open(datasets) as file:
            data = json.load(file)

        for id, item in data.items():
            sentences = item["context"]
            if item["evidence"] != None:
              for sentence in sentences:
                evidence = item['evidence'] in sentence or sentence in item['evidence']
                if evidence:
                  data_sample_pos = {}
                  # data_sample_pos['id'] = id
                  data_sample_pos['claim'] = item['claim']
                  data_sample_pos['sentence'] = sentence
                  data_sample_pos['evidence'] = item['evidence']
                  data_sample_pos['evidence_label'] = 1
                  data_sample_pos["verdict"] = item["verdict"]
                  data_sample_pos['domain'] = item['domain']
                  self.samples_pos.append(data_sample_pos)

                else:
                  data_sample_neg = {}
                  # data_sample_neg['id'] = id
                  data_sample_neg['claim'] = item['claim']
                  data_sample_neg['sentence'] = sentence
                  data_sample_neg['evidence'] = item['evidence']
                  data_sample_neg['evidence_label'] = 0
                  data_sample_neg["verdict"] = item["verdict"]
                  data_sample_neg['domain'] = item['domain']
                  self.samples_neg.append(data_sample_neg)


    def to_dataframe(self):
        return pd.DataFrame(self.samples_pos).sample(frac=1, random_state=3), pd.DataFrame(self.samples_neg).sample(frac=1, random_state=3)

    def save_result (self, save_path):
      with open(f"{save_path}/converted_data_train_tocken_pos.json", "w", encoding="utf-8") as file:
        json.dump(self.samples_pos , file, ensure_ascii=False)
      with open(f"{save_path}/converted_data_train_tocken_neg.json", "w", encoding="utf-8") as file:
        json.dump(self.samples_neg , file, ensure_ascii=False)


In [18]:
dat_path = "/content/drive/MyDrive/pRoBERTa/data/converted_data_train_tocken.json"

In [19]:
a = LoadDataFromJson(dat_path)

In [20]:
save_path = "/content/drive/MyDrive/pRoBERTa/data/data"

In [21]:
# a.save_result(save_path)

In [22]:
pos, neg = a.to_dataframe()

In [23]:
pos

,claim,sentence,evidence,evidence_label,verdict,domain
7284,trước papi chưa có ai bị gấu_vồ giết ở ý trong...,papi là người đầu_tiên ở italy bị gấu_vồ chết ...,papi là người đầu_tiên ở italy bị gấu_vồ chết ...,1,SUPPORTED,thegioi
8493,thông_cáo bày_tỏ mong_muốn được chào_đón con t...,chúng_tôi hy_vọng sẽ sớm lại được chào_đón con...,chúng_tôi hy_vọng sẽ sớm lại được chào_đón con...,1,SUPPORTED,thegioi
9298,khu đất trên ở xã phạm_văn hai huyện bình_chán...,khu đất trên ở xã phạm_văn hai huyện bình_chán...,khu đất trên ở xã phạm_văn hai huyện bình_chán...,1,SUPPORTED,thoisu
14080,nhóm nghiên_cứu nghi_ngờ đây là kết_quả từ thi...,nhóm nghiên_cứu nghi_ngờ đây là kết_quả từ việ...,nhóm nghiên_cứu nghi_ngờ đây là kết_quả từ việ...,1,REFUTED,khoahoc
8585,một trận giao_hữu bóng_đá đã diễn ra giữa thủy...,“ chúng_tôi hôm_nay rất vui khi thủy_thủ của t...,“ chúng_tôi hôm_nay rất vui khi thủy_thủ của t...,1,SUPPORTED,thegioi
...,...,...,...,...,...,...
9160,đất lành chim đậu,dân_làng đã thử nhổ về trồng nhưng do môi_trườ...,dân_làng đã thử nhổ về trồng nhưng do môi_trườ...,1,SUPPORTED,thoisu
9859,các đơn_vị được yêu_cầu trung_thực,các đơn_vị thưởng hợp_đồng được yêu_cầu công_k...,các đơn_vị thưởng hợp_đồng được yêu_cầu công_k...,1,SUPPORTED,thoisu
11513,đến bây_giờ có hơn 30 chuột bạch_nuôi trong hơ...,đến nay viện có hơn 30 chuột bạch_nuôi trong h...,đến nay viện có hơn 30 chuột bạch_nuôi trong h...,1,SUPPORTED,khoahoc
1688,học_sinh được khuyến_khích thử_nghiệm cái mới ...,học_sinh tại phenikaa được tự nghiên_cứu đào_s...,học_sinh tại phenikaa được tự nghiên_cứu đào_s...,1,SUPPORTED,giaoduc


In [24]:
neg

,claim,sentence,evidence,evidence_label,verdict,domain
180739,một trong những địa_phương có diện_tích nuôi c...,ở các tỉnh miền tây nông_dân có_thể đào ao hoặ...,xã tân_thành là địa_phương có diện_tích nuôi c...,0,SUPPORTED,thoisu
189625,những sự_kiện nổi_bật nhân_dịp kỷ_niệm 50 năm ...,theo thông_tin được hải_quân italia công_bố fr...,“ chuyến thăm của tàu morosini là một trong nh...,0,SUPPORTED,thegioi
76501,trình_độ tiếng anh của người việt_nam được cải...,họ xì_xào rằng tại_sao em lại hẹn_hò với một n...,việt_nam ngày_càng thu_hút nhiều người nước_ng...,0,SUPPORTED,thegioi
14781,việc lắp_đặt hyperloop giúp các khu_vực ở trun...,phát_triển kỹ_thuật cần_thiết cho hệ_thống hyp...,dự_án như_vậy cũng tác_động tích_cực tới xã_hộ...,0,SUPPORTED,khoahoc
68592,ít_nhất 6 người chết và hơn 30 người bị_thương...,các nhà_chức_trách cho biết bão_bụi ập tới nha...,cảnh_sát cho biết ít_nhất 6 người chết và hơn ...,0,SUPPORTED,khoahoc
...,...,...,...,...,...,...
11261,mô_hình ba chiều của khối_u ác_tính gần tim củ...,rất nhiều ý_tưởng sẽ không_thể thành_công nếu ...,theo yêu_cầu của bác_sĩ nó đã chuyển hình_ảnh ...,0,SUPPORTED,khoahoc
48056,drh clinic cho biết đơn_vị đã và sẽ đầu_tư hơn...,đó là các bệnh_lý mụn_rỗ nám biến_chứng với cá...,trong tương_lai drh clinic cam_kết sẽ tiếp_tục...,0,SUPPORTED,suckhoe
77049,nhiều tài_liệu mật chuẩn_bị cho các quan_chức ...,đại_tá yuriy ihnat_phát_ngôn_viên của lực_lượn...,đánh_giá này nằm trong số hàng chục tài_liệu m...,0,SUPPORTED,thegioi
198296,nhiều năm bán xe_máy ông minh chủ một đại_lý ở...,sáng 18/12 chị huỳnh_thị xuân_thảo 33 tuổi lau...,nhiều năm kinh_doanh xe_máy ông minh chủ một đ...,0,SUPPORTED,thoisu


In [25]:
neg.drop_duplicates(inplace=True)

In [26]:
neg.duplicated().sum()


0

In [27]:
filtered_df_pos = pos.loc[(pos['sentence'].apply(lambda x: len(x.split())) < 3)]
filtered_df_pos


,claim,sentence,evidence,evidence_label,verdict,domain
12212,vợ anh đánh_giá rằng ba phần cơm cho cả gia_đì...,huỳnh_nhi,huỳnh_nhi,1,SUPPORTED,dulich
8939,người bốc_xếp được trả 1 ngàn đồng cho mỗi cuộ...,hoàng nam,hoàng nam,1,SUPPORTED,thoisu
10397,ngoài_ra hiếu_luyện viết_luận mỗi tuần để có_t...,,ngoài_ra hiếu_tập viết_luận mỗi tuần để có_thể...,1,SUPPORTED,giaoduc
7475,biden thường cố_gắng hoàn_thành công_việc mà ô...,,mỗi khi joe biden được bảo rằng ông không_thể ...,1,SUPPORTED,thegioi
6193,trong đó nơi có công_trình chống xói_lở bờ biể...,an_minh,an_minh,1,SUPPORTED,thoisu
...,...,...,...,...,...,...
14283,vì_vậy như châu_muốn nhờ độc_giả tư_vấn giúp n...,xin cám_ơn,vì_vậy tôi muốn nhờ độc_giả tư_vấn giúp nên dù...,1,REFUTED,dulich
13252,các cô các chú không được đem hiểu_biết khoa_h...,hải_minh,hải_minh,1,REFUTED,khoahoc
5703,trước_đây vì hồ trị_an rộng 320 ha nên ngập mê...,phước tuấn,phước tuấn,1,SUPPORTED,thoisu
11152,thị_trấn cổ đã có 1300 tuổi và mang đến một vẻ...,tổng_hợp,tổng_hợp,1,SUPPORTED,dulich


In [28]:
filtered_df_neg = neg.loc[(neg['sentence'].apply(lambda x: len(x.split())) < 3)]
filtered_df_neg

,claim,sentence,evidence,evidence_label,verdict,domain
82526,tình_trạng biến_chứng của 3 người còn lại nhẹ hơn,minh an,những người còn lại cũng bị biến_chứng dị_ứng ...,0,SUPPORTED,suckhoe
4793,yếu_tố thẩm_mỹ đúng nguyên_tắc của một bài_văn...,vn,nữ giáo_viên tại hệ_thống giáo_dục hocmai đưa ...,0,SUPPORTED,giaoduc
224459,minh_dũng cho biết tú chưa từng trốn trong các...,thanh hằng,minh_dũng cho biết tú chưa từng vắng_mặt trong...,0,SUPPORTED,giaoduc
190067,mối liên_kết mạnh_mẽ về văn_hóa giữa việt_nam ...,bảo đức,sự yêu_mến rộng_rãi dành cho yoga tại việt_nam...,0,SUPPORTED,thegioi
231882,ông kiến nói rằng cháo ở đây được nấu theo kiể...,trần quỳnh,theo ông cháo ở đây nấu đúng kiểu người hoa_tr...,0,SUPPORTED,dulich
...,...,...,...,...,...,...
294107,geos không_thể phục_vụ nghiên_cứu hệ_thống đại...,2,geos có_thể phục_vụ nghiên_cứu hệ_thống đại_dư...,0,REFUTED,khoahoc
216469,nhiều oto né phí,hoàng nam,ông nguyễn_văn_sáu trưởng phòng kỹ_thuật công_...,0,SUPPORTED,thoisu
299446,đặc_biệt quảng_trường nhật_thực cũng nổi_bật v...,doãn phong,đặc_biệt khu_vực này cũng nổi_bật với vườn tượ...,0,REFUTED,dulich
97769,silicon dễ gây biến_chứng khi ngấm vào các mô_...,minh an,silicon lỏng dễ ngấm vào các mô_mỡ và mạch_máu...,0,SUPPORTED,suckhoe


In [29]:
pos = pos.drop(pos[pos.isin(filtered_df_pos.to_dict('list')).all(1)].index)

In [30]:
neg = neg.drop(neg[neg.isin(filtered_df_neg.to_dict('list')).all(1)].index)


In [31]:
dev_pos = pos.head(1400)
pos = pos.drop(dev_pos.index)
dev_neg = neg.head(1400*2)
neg = neg.drop(dev_neg.index)
df_eval = pd.concat([dev_pos, dev_neg])
df_train = pd.concat([pos, neg])

In [32]:
# df_train.reset_index(drop=True, inplace=True)
# df_eval.reset_index(drop=True, inplace=True)

In [33]:
# df_eval.to_json(f"{save_path}/dev.json")

In [34]:
# df_train.to_json(f"{save_path}/train.json")


In [35]:
row = df_train.iloc[12212]["claim"]
print(row)


quần_đảo caribe và mexico bán_đảo yucatán canada có nhiều lợn hoang


In [36]:
a = " Hàng ốc sẽ bắt, (đầu mở) bán từ 15h đến khoảng, 21h hàng ngày"

In [37]:
b = preprocess_text(a)
c = tonghop(a)

In [38]:
b

'hàng ốc sẽ bắt (đầu mở) bán từ 15h đến khoảng 21h hàng ngày'

In [39]:
c

'Hàng_ốc sẽ bắt , ( đầu mở ) bán từ 15 h đến khoảng , 21 h hàng ngày'